# Parser Prototype

In [ ]:
from lark import Lark

grammar = r"""
requirements    : ">" requirement (">" requirement)*

requirement     : req_name expression "---" error_desc

req_name        : ESCAPED_STRING
error_desc      : ESCAPED_STRING

?expression     : iff_expr

?iff_expr       : (implies_expr "iff")? implies_expr

?implies_expr   : (and_expr "implies")? and_expr

?and_expr       : (or_expr "and")* or_expr

?or_expr        : (not_expr "or")* not_expr

?not_expr       : "not" not_expr -> negation
                | quantification

?quantification : "exists" bound_consts "(" expression ")" -> exists
                | "forall" bound_consts "(" expression ")" -> forall
                | "(" expression ")"
                | property

?property       : CONST "has association" RELATIONSHIP CONST    -> association_expr
                | CONST "has attribute" RELATIONSHIP VALUE      -> attribute_expr
                | const_or_class "is" const_or_class            -> equality
                | const_or_class "is not" const_or_class        -> inequality

bound_consts    : [CONST ("," CONST)*]

const_or_class  : "class" CLASS
                | CONST 

EQUALITY_OP     : "is"
                | "is not"

// regex: /[a-zA-Z]+_[a-zA-Z]+::[a-zA-Z]+/
//RELATIONSHIP: (LCASE_LETTER) ("_"|"::"|LETTER|DIGIT)*
RELATIONSHIP: /[^\W\d_]+\.[^\W\d_]+->[^\W\d_]+/

//CLASS: (LCASE_LETTER) ("_"|LETTER|DIGIT)*
CLASS: /[^\W\d_]+\.[^\W\d_]+/

// Const must start with lowercase letter
CONST: (LCASE_LETTER) ("_"|LETTER|DIGIT)*
// Value must start with uppercase letter
VALUE: (UCASE_LETTER) ("_"|LETTER|DIGIT)*

// Comment: python/sh style
COMMENT: /#[^\n]*/

%import common.ESCAPED_STRING
%import common.NEWLINE
%import common.LETTER
%import common.DIGIT
%import common.LCASE_LETTER
%import common.UCASE_LETTER
%import common.WS
%ignore WS
%ignore COMMENT

"""

In [ ]:
# vm, iface = get_consts(smtsorts, ["vm", "iface"])
# return And(
#     smtenc.element_class_fun(vm) == smtenc.classes["infrastructure_VirtualMachine"],
#     Not(
#         Exists(
#             [iface],
#             ENCODINGS.association_rel(vm, smtenc.associations["infrastructure_ComputingNode::ifaces"], iface)
#         )
#     )
# )

expr_to_parse = r"""
>   "example requirement to test"
    # Expr to parse
    not ( 
        vm is class infrastructure.VirtualMachine
        and
        vm is not class infrastructure.Storage
        or
        vm is not class infrastructure.Storage
        implies
        vm is class infrastructure.Storage
    )
    iff
    not exists iface, apple (
        forall orange (
            vm has association infrastructure.ComputingNode->ifaces iface
            or
            vm has association infrastructure.ComputingNode->ifaces iface
        )
        and
        vm has attribute infrastructure.ComputingNode->os Os1
    )
    ---
    "Virtual Machine {vm} has no iface"
"""

In [ ]:
parser = Lark(grammar, start="requirements", parser="lalr", debug=True)
tree = parser.parse(expr_to_parse)

print(tree.pretty())

We need the `ModelChecker` to import `SMTEncodings` and `SMTSorts` in order to create our Z3 constants programmatically.

Now the model checker should expose the *intermediate model checker* which should provide us with those two collections.

In [ ]:
from mc_openapi.doml_mc import ModelChecker, DOMLVersion
from mc_openapi.doml_mc.imc import IntermediateModelChecker

# Import DOMLX as bytes
doml_document_path = "../../tests/doml/faas.domlx"
with open(doml_document_path, "rb") as xmif:
    doml_xmi = xmif.read()

model_checker = ModelChecker(doml_xmi, DOMLVersion.V2_0)

intermediate_model_checker = IntermediateModelChecker(
    model_checker.metamodel,
    model_checker.inv_assoc,
    model_checker.intermediate_model
)

ENCODINGS =  intermediate_model_checker.smt_encoding
SORTS = intermediate_model_checker.smt_sorts

assert ENCODINGS and SORTS

The parser will now produce a Z3 expression to evaluate.

In [ ]:
from typing import Callable
from mc_openapi.doml_mc.imc import Requirement, RequirementStore, SMTEncoding, SMTSorts, IntermediateModel
from mc_openapi.dsl_parser.utils import RefHandler, VarStore
from mc_openapi.doml_mc.error_desc_helper import get_user_friendly_name
from lark import Transformer
from z3 import ExprRef, Solver, Not, And, Implies, Or, Xor, Exists, ForAll

class Parser:
    def __init__(self, grammar: str = grammar):
        self.parser = Lark(grammar, start="requirements")

    def parse(self, input: str) -> RequirementStore:
        self.tree = self.parser.parse(input)

        const_store = VarStore()

        transformer = DSLTransformer(const_store)

        return RequirementStore(transformer.transform(self.tree))

class DSLTransformer(Transformer):
    # These callbacks will be called when a rule with the same name
    # is matched. It starts from the leaves.
    def __init__(self, const_store: VarStore, visit_tokens: bool = True) -> None:
        super().__init__(visit_tokens)
        self.const_store = const_store

    def __default__(self, data, children, meta):
        return children

    # start
    def requirements(self, args) -> list[Requirement]:
        return args

    def requirement(self, args) -> Requirement:
        name: str = args[0]
        expr: Callable[[SMTEncoding, SMTSorts], ExprRef] = args[1]
        errd: Callable[[Solver, SMTSorts, IntermediateModel, int], str] = args[2]
        return Requirement(
            expr,
            name.lower().replace(" ", "_"),
            name,
            lambda solver, sorts, model: errd(solver, sorts, model, self.const_store.get_index_and_push())
        )

    def req_name(self, args) -> str:
        return str(args[0].value.replace('"', ''))

    def bound_consts(self, args):
        const_names = list(map(lambda arg: arg.value, args))
        for name in const_names:
            self.const_store.use(name)
            self.const_store.quantify(name)
        return lambda _, sorts: RefHandler.get_consts(const_names, sorts)

    def negation(self, args):
        return lambda enc, sorts: Not(args[0](enc, sorts))

    def iff_expr(self, args):
        return lambda enc, sorts: args[0](enc, sorts) == args[1](enc, sorts)
    
    def implies_expr(self, args):
        return lambda enc, sorts: Implies(args[0](enc, sorts), args[1](enc, sorts))

    def and_expr(self, args):
        return lambda enc, sorts: And(args[0](enc, sorts), args[1](enc, sorts))

    def or_expr(self, args):
        return lambda enc, sorts: Or(args[0](enc, sorts), args[1](enc, sorts))

    def exists(self, args):
        return lambda enc, sorts: Exists(args[0](enc, sorts), args[1](enc, sorts))

    def forall(self, args):
        return lambda enc, sorts: ForAll(args[0](enc, sorts), args[1](enc, sorts))

    def association_expr(self, args):
        self.const_store.use(args[0].value)
        self.const_store.use(args[2].value)
        return lambda enc, sorts: RefHandler.get_association_rel(
            enc,
            RefHandler.get_const(args[0].value, sorts),
            RefHandler.get_association(enc, args[1].value),
            RefHandler.get_const(args[2].value, sorts)
        )

    def attribute_expr(self, args):
        self.const_store.use(args[0].value)
        return lambda enc, sorts: RefHandler.get_attribute_rel(
            enc,
            RefHandler.get_const(args[0].value, sorts),
            RefHandler.get_attribute(enc, args[1].value),
            RefHandler.get_value(args[2].value, sorts)
        )

    def equality(self, args):
        return lambda enc, sorts: args[0](enc, sorts) == args[1](enc, sorts)

    def inequality(self, args):
        return lambda enc, sorts: args[0](enc, sorts) != args[1](enc, sorts)

    def const_or_class(self, args):
        if args[0].type == "CONST":
            self.const_store.use(args[0].value)
            return lambda enc, sorts: RefHandler.get_element_class(enc, RefHandler.get_const(args[0].value, sorts))
        elif args[0].type == "CLASS":
            return lambda enc, _: RefHandler.get_class(enc, args[0].value)
    
    def error_desc(self, args):
        def err_callback(
            solver: Solver,
            sorts: SMTSorts,
            intermediate_model: IntermediateModel,
            index
        ) -> str:
            msg: str = args[0].value.replace('"', '')
            consts_name = self.const_store.get_free_vars(index)
            consts = RefHandler.get_consts(consts_name, sorts)
            model = solver.model()
            for const in consts:
                name = get_user_friendly_name(intermediate_model, model, const)
                msg = msg.replace("{" + str(const) + "}", f"'{name}'")
            return msg
        return err_callback



In [ ]:
from pprint import pprint
from z3 import Not, And, Or, Xor, Implies, Exists, ForAll

# from mc_openapi.dsl_parser.parser import Parser

parser = Parser(grammar)

reqs_store = parser.parse(expr_to_parse)

pprint(reqs_store.get_all_requirements())

In [ ]:
for req in reqs_store.get_all_requirements():
    print(req.assert_callable(ENCODINGS, SORTS))